Idee:
E-Sport
- Wie oft wurde E-Sport über die letzten Jahre in den News erwähnt
    - Einfluss von Corona, Einfluss von "Großereignissen" - League of Legends Worlds, CS:GO Major Turniere
- Wie oft wird das auf welcher Seite erwähnt
- Wie oft wurde welches Spiel erwähnt (CS:GO, League of Legends, Fortnite, Dota, PUBG etc.)

In [1]:
import lib_py.scraper as sc
import lib_py.etl_new as etl
import pandas as pd
import os
import sqlite3

```
select * from wordcount where word LIKE '%sport%' order by date
select * from wordcount where word LIKE '%sport%' order by paper, date
select * from wordcount where word LIKE '%gaming%' order by date
select * from wordcount where word LIKE '%gaming%' order by paper, date
select sum(count), word from wordcount group by word
select sum(count), filter_word, other_word from combinationcount group by filter_word, other_word
select * from combinationcount where filter_word = 'league' AND other_word = 'legends' order by date
```

```
SELECT sum(count)
	,filter_word
	,other_word
FROM combinationcount
WHERE filter_word != 'cs'
	AND (filter_word != 'league' and other_word != 'legends')
	AND filter_word != 'fifa'
GROUP BY filter_word
	,other_word
ORDER BY sum(count) DESC
```

In [ ]:
etl.main()

In [2]:
etl.process_one_logfile("output/data-lake/2023-01-01.csv")

[INFO] Processing  output/data-lake/2023-01-01.csv


In [ ]:
newspaper = {'Unnamed: 0': 0,
             'name': 'sz',
             'date': '2023-03-31',
             'file_name': 'data-lake/2023-03-31-sz.html',
             'status': 200,
             'original_url': 'https://www.sueddeutsche.de/',
             'final_url': 'https://www.sueddeutsche.de/',
             'encoding': 'UTF-8'}
result = etl.process_wrapper(newspaper)

In [ ]:
print(type(result[0]))
result[0]

In [ ]:
print(type(result[1]))
result[1]

In [ ]:
type(result[1]['count'])

In [ ]:
type(result[0]['count'])

In [ ]:
SQL_PATH = os.path.join("output", "dwh.sqlite3")

collection_count = []
collection_combiniations = []

if type(result[0]).__name__ == "DataFrame":
    collection_count.append(result[0])
if type(result[1]).__name__ == "DataFrame":
    collection_combiniations.append(result[1])

data_count = pd.concat(collection_count, axis=0)
data_combiniations = pd.concat(collection_combiniations, axis=0)

def convert_to_string(lst):
    return ", ".join(lst)

data_combiniations['other_word'] = data_combiniations['other_word'].apply(convert_to_string)

connection = sqlite3.connect(SQL_PATH)
data_count.to_sql("wordcount", connection, index=False, if_exists="append", dtype={"count":"INTEGER"})
data_combiniations.to_sql("combinationcount", connection, index=False, if_exists="append", dtype={"count":"INTEGER"})